## Model Training

In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/home/smruti/rice-leaf-disease-detection'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from rldd.constants import *
from rldd.utils import read_yaml, create_directories
import tensorflow as tf

2024-10-29 16:19:40.598089: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 16:19:40.639177: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "dataset")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [7]:
import time

In [8]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    @property
    def _create_early_stopping_callback(self):
        return tf.keras.callbacks.EarlyStopping(
            patience=10,
            restore_best_weights=True
        )
    


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self._create_early_stopping_callback
        ]


In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import math

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        data_dir = self.config.training_data
        seed = 123

        # Splitting data and creating training and validation datasets
        self.train_ds = tf.keras.utils.image_dataset_from_directory(
            data_dir,
            validation_split=0.1,
            subset="training",
            seed=seed,
            image_size=(self.config.params_image_size[0], self.config.params_image_size[1]),
            batch_size=self.config.params_batch_size
        )

        self.validation_ds = tf.keras.utils.image_dataset_from_directory(
            data_dir,
            validation_split=0.1,
            subset="validation",
            seed=seed,
            image_size=(self.config.params_image_size[0], self.config.params_image_size[1]),
            batch_size=self.config.params_batch_size
        )

        # Normalizing Pixel Values for Training and Validation Datasets
        self.train_ds = self.train_ds.map(lambda x, y: (x / 255.0, y))
        self.validation_ds = self.validation_ds.map(lambda x, y: (x / 255.0, y))

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):

        self.model.fit(
            self.train_ds,
            epochs=self.config.params_epochs,
            validation_data=self.validation_ds,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-10-29 16:19:42,291: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-29 16:19:42,294: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-29 16:19:42,294: INFO: common: created directory at: artifacts]
[2024-10-29 16:19:42,297: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2024-10-29 16:19:42,297: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2024-10-29 16:19:42,299: INFO: common: created directory at: artifacts/training]


2024-10-29 16:19:42.494369: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-29 16:19:42.499824: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-29 16:19:42.499859: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-29 16:19:42.501564: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-29 16:19:42.501596: I external/local_xla/xla/stream_executor

Found 10407 files belonging to 10 classes.
Using 9367 files for training.
Found 10407 files belonging to 10 classes.
Using 1040 files for validation.
Epoch 1/100


I0000 00:00:1730218795.685006   79605 service.cc:145] XLA service 0x7f7434003490 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730218795.685151   79605 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-10-29 16:19:56.164382: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-29 16:19:57.790112: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1730218822.039084   79605 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


293/293 ━━━━━━━━━━━━━━━━━━━━ 111s 262ms/step - accuracy: 0.5190 - loss: 1.4553 - val_accuracy: 0.7683 - val_loss: 0.7863
Epoch 2/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 30s 101ms/step - accuracy: 0.7648 - loss: 0.7143 - val_accuracy: 0.8413 - val_loss: 0.5108
Epoch 3/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 30s 101ms/step - accuracy: 0.8143 - loss: 0.5610 - val_accuracy: 0.8721 - val_loss: 0.4013
Epoch 4/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 30s 102ms/step - accuracy: 0.8519 - loss: 0.4661 - val_accuracy: 0.8913 - val_loss: 0.3462
Epoch 5/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 30s 102ms/step - accuracy: 0.8683 - loss: 0.3932 - val_accuracy: 0.8962 - val_loss: 0.3229
Epoch 6/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 30s 102ms/step - accuracy: 0.8910 - loss: 0.3319 - val_accuracy: 0.9106 - val_loss: 0.2798
Epoch 7/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 29s 99ms/step - accuracy: 0.9008 - loss: 0.2878 - val_accuracy: 0.9067 - val_loss: 0.2880
Epoch 8/100
293/293 ━━━━━━━━━━━━━━━━━━━━ 29s 99ms/step - accuracy: 0.9174 - loss: 0.2523